# Code Interpreter

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool,FilePurpose, MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [2]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup the Code Interpreter

In [3]:
import os

file_directory = "../data/population"

# List all files in the directory
try:
    filenames = os.listdir(file_directory)
    print(filenames)
except FileNotFoundError:
    print(f"Directory '{file_directory}' not found.")

# Get the full path of a file
def get_filepath_for_filename(filename: str) -> str:
    base_directory = file_directory
    return os.path.join(base_directory, filename)

# Upload files to the project client
file_ids: list[str] = []
for path in [get_filepath_for_filename(filename) for filename in filenames]:
    with open(path, "rb") as file:
        file = project_client.agents.files.upload_and_poll(file_path=path, purpose=FilePurpose.AGENTS)
        print(f"Uploaded file, file ID: {file.id}")
        file_ids.append(file.id)

code_interpreter = CodeInterpreterTool(file_ids=file_ids)

['PopulationByAdmin1.csv', 'PopulationByCountry.csv']
Uploaded file, file ID: assistant-VLKPbJZa3mhjiq1B35Mj74
Uploaded file, file ID: assistant-RMoQRRtKKwn5heE2H81vgz


## Create the agent

In [4]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-codeinterpreter-agent",  # Name of the agent
    instructions="You are a helpful agent that can write Python code to answer questions from a dataset",  # Instructions for the agent
    tools=code_interpreter.definitions,  # Tools available to the agent
    tool_resources=code_interpreter.resources,  # Resources for the tools
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_uRgsn0bG70XbRYXNZLaSyRMa


## Create thread

In [5]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_NaxnznSFJyasRSy7xGbagP9T


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [6]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":       
        last_msg = project_client.agents.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            # Extract and print the text value
            for item in last_msg['content']:
                if item['type'] == 'text':
                    print("Agent Response:", item['text']['value'])

            # Save every image file in the message
            for img in last_msg.image_contents:
                file_id = img.image_file.file_id
                file_name = f"{file_id}_image_file.png"
                project_client.agents.files.save(file_id=file_id, file_name=file_name)
                print(f"Saved image file to: {file_name}")

            # Print details of every file-path annotation
            for ann in last_msg.file_path_annotations:
                print("File Paths:")
                print(f"  Type: {ann.type}")
                print(f"  Text: {ann.text}")
                print(f"  File ID: {ann.file_path.file_id}")
                print(f"  Start Index: {ann.start_index}")
                print(f"  End Index: {ann.end_index}")

## Run the agent

In [7]:
user_input = "What country has the 3rd highest population?"
run_agent(user_input)

Created message, ID: msg_jWgUyLLgwOYZkAZM2wuOSF74
Run finished with status: completed
Agent Response: The country with the 3rd highest population is the United States (US) with a population of 661,281,782.


In [8]:
user_input = """
  Give me a light green column chart in descending order of the top 10 countries with the highest population. 
  Place the population values on top of each chart
"""
run_agent(user_input)

Created message, ID: msg_0Z08vrRpwyEHXkLDosOSrtab
Run finished with status: completed
Agent Response: Here is the light green column chart showing the top 10 countries with the highest population, with the population values displayed on top of each bar.
Saved image file to: assistant-JQUofsAFiF4PTnVoNE1hgD_image_file.png


In [ ]:
user_input = """
  Provide me a light blue column chart in descending order of the top 5 provinces in Japan with the highest population. 
  Place the population values on top of each chart
"""
run_agent(user_input)

## Cleanup resources

In [ ]:
# Cleanup resources

# Delete the files after use
for file_id in file_ids:
  project_client.agents.files.delete(file_id)
  print("Deleted file:", file_id)

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
